In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather

In [2]:
import glob
import os

In [3]:
#path to data
data = "/home/mlopez/EXEC/Observed data/NRCAN_obs_tg_mean_annual.nc"

# Create DF of observed data

In [6]:
dfobstg = pcdf.load_as_df(data)

In [8]:
dfobstg.reset_index()

,lat,lon,time,tg_mean
0,83.125000,-74.541672,1950-01-01,251.429901
1,83.125000,-74.541672,1951-01-01,252.994797
2,83.125000,-74.541672,1952-01-01,253.964935
3,83.125000,-74.541672,1953-01-01,253.616013
4,83.125000,-74.541672,1954-01-01,253.628616
...,...,...,...,...
16464827,41.708332,-82.208336,2009-01-01,282.584686
16464828,41.708332,-82.208336,2010-01-01,283.870300
16464829,41.708332,-82.208336,2011-01-01,284.144257
16464830,41.708332,-82.208336,2012-01-01,285.570129


# Transform temp to Celsius

In [9]:
dfC = dfobstg.copy()
dfC["tg_mean"] = dfC["tg_mean"] -273.15
    

# Define periods of time: 

In [10]:
year_groups = {y:0 for y in range(1980,2011)}
#Get the mean of every period
dfC2 = dfC.reset_index()
dfp = dfC2.groupby([dfC2.time.dt.year.map(year_groups), "lat","lon"]).mean()

In [11]:
dfp.reset_index()

,time,lat,lon,tg_mean
0,0.0,41.708332,-82.791672,10.039598
1,0.0,41.708332,-82.708336,9.940871
2,0.0,41.708332,-82.625000,9.872470
3,0.0,41.708332,-82.541672,9.838626
4,0.0,41.708332,-82.458336,9.807738
...,...,...,...,...
257258,0.0,83.125000,-69.958336,-19.014936
257259,0.0,83.125000,-69.875000,-18.887545
257260,0.0,83.125000,-69.791672,-18.915213
257261,0.0,83.125000,-69.708336,-18.884167


# Historic Period

485,881/3 = 161,960. ---- instead 333,469 rows × 4 columns

In [12]:
df_h = dfp.query("time==0.0")
df_h.reset_index()

,time,lat,lon,tg_mean
0,0.0,41.708332,-82.791672,10.039598
1,0.0,41.708332,-82.708336,9.940871
2,0.0,41.708332,-82.625000,9.872470
3,0.0,41.708332,-82.541672,9.838626
4,0.0,41.708332,-82.458336,9.807738
...,...,...,...,...
257258,0.0,83.125000,-69.958336,-19.014936
257259,0.0,83.125000,-69.875000,-18.887545
257260,0.0,83.125000,-69.791672,-18.915213
257261,0.0,83.125000,-69.708336,-18.884167


## Import dataframe with polygons

In [13]:
dfpolyshape = from_geofeather('Grid-TerritoiresGuides.feather')

/home/mlopez/EXEC/anaconda3/envs/python37-2/lib/python3.8/site-packages/geofeather/core.py:85: UserWarning: Grid-TerritoiresGuides.feather.crs coordinate reference system file is missing. No crs will be set for this GeoDataFrame.
  warnings.warn(


In [14]:
dfpolyshape

,lat,lon,TER_GUIDE,geometry
0,52.957191,-67.712730,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
1,52.957191,-67.629402,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."
2,52.957191,-67.546066,6opqr,"POLYGON ((-67.56730 52.91553, -67.57974 52.915..."
3,52.873859,-67.796059,6opqr,"POLYGON ((-67.75439 52.89329, -67.75439 52.832..."
4,52.873859,-67.712730,6opqr,"POLYGON ((-67.75439 52.83219, -67.75439 52.893..."
...,...,...,...,...
33193,45.041668,-72.208336,2c,"POLYGON ((-72.25000 45.00450, -72.25000 45.083..."
33194,45.041668,-72.125000,2c,"POLYGON ((-72.16666 45.00508, -72.16666 45.083..."
33195,45.041668,-72.041672,2c,"POLYGON ((-72.08334 45.00564, -72.08334 45.083..."
33196,45.041668,-71.958336,2c,"POLYGON ((-72.00000 45.00667, -72.00000 45.083..."


## Merge data with mask 

In [15]:
dftghTG = pd.merge(df_h, dfpolyshape, on=["lat","lon"])

In [16]:
dftghTG

,lat,lon,tg_mean,TER_GUIDE,geometry
0,44.958332,-74.625000,7.262820,1a,"POLYGON ((-74.66365 45.00000, -74.58334 45.000..."
1,44.958332,-74.541672,7.225025,1a,"POLYGON ((-74.58334 44.99872, -74.58334 45.000..."
2,44.958332,-74.458336,7.186191,1a,"POLYGON ((-74.50000 44.99779, -74.50000 45.000..."
3,44.958332,-74.375000,7.153564,1a,"POLYGON ((-74.41666 44.99517, -74.41666 45.000..."
4,44.958332,-74.291672,7.095845,1a,"POLYGON ((-74.33334 44.99213, -74.33334 45.000..."
...,...,...,...,...,...
16559,52.875000,-67.625000,-2.909539,6opqr,"POLYGON ((-67.66666 52.83333, -67.66666 52.916..."
16560,52.875000,-67.541672,-2.857326,6opqr,"POLYGON ((-67.58334 52.83333, -67.58334 52.911..."
16561,52.958332,-67.708336,-3.045003,6opqr,"POLYGON ((-67.66667 52.93419, -67.66667 52.916..."
16562,52.958332,-67.625000,-2.945693,6opqr,"POLYGON ((-67.66666 52.91667, -67.66666 52.934..."


In [17]:
dftghTG.TER_GUIDE.value_counts()

6cdefg    2361
6opqr     1609
6ab       1416
5bcd      1237
4bc       1048
6hi       1026
6j         949
6mn        925
3ab        652
5a         550
6kl        512
5ef        399
4f         394
2b         388
3d         365
3c         355
4de        355
5g         350
5hi        341
1a         340
4gh        301
5jk        245
2c         165
2a         142
4a         139
Name: TER_GUIDE, dtype: int64

In [ ]:
TG4a = dftg11TG[dftg11TG.TER_GUIDE == "4a"]
TG4a

In [ ]:
TG4a.to_csv("/home/mlopez/EXEC/Tg_csv/TG4a11.csv")

# ---- Create GeoJson file ----

In [18]:
geometry = dftghTG["geometry"]

In [20]:
crs = {'init': "epsg:4326"}
gdf = GeoDataFrame(dftghTG, crs=crs, geometry=geometry)

In [21]:
gdf.to_file("TGobstg.json", driver="GeoJSON")